In [1]:
import pandas as pd
import numpy as np

import requests
import json

import sys
sys.path.insert(0,'/home/caden/Programming/info')
from Keys import Keys

In [27]:
header_df = pd.read_csv('../../data/header.csv')
header_df.columns

Index(['device_id', 'lon', 'lat', 'STATEFP', 'COUNTYFP', 'TRACTCE',
       'BLKGRPCE'],
      dtype='object')

In [29]:
[str(county) for county in header_df.COUNTYFP.unique()]

['339', '201', '15', '167', '157', '71', '39', '291', '473']

In [109]:
host = 'https://api.census.gov/data'
years = ['2019','2020','2021']
dataset_acronym = 'acs/acs5'
api_key = str(Keys.CensusAPI())

# B19083_001E - Gini Index (removed bc probably too colinear w per capita income)
# B19301_001E - Per Capita Income
# B19013_001E - Median Household Income

var_list = "NAME"
income = ['B19013_001E', 'B19301_001E']
race = [f'B02001_00{i}E' for i in range(1,8)]
age = 'B01002_001E'

var_list = [var_list,age] + income + race
var_list = ','.join(var_list)

state = '48'
counties = [str(county) for county in header_df.COUNTYFP.unique()]

In [61]:
var_list

'NAME,B01002_001E,B19013_001E,B19301_001E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E'

In [116]:
county_strings = []

for county in counties:
    l = 3 - len(county)
    county = (l * '0') + county
    county_strings.append(county)

data = []
county_strings

['339', '201', '015', '167', '157', '071', '039', '291', '473']

In [117]:
for year in years: 
    for county in county_strings:
        print(year, state, county)
        query_url = f'https://api.census.gov/data/{year}/acs/acs5?get={var_list}&for=block+group:*&in=state:48%20county:{county}%20tract:*&key={api_key}'
        response = requests.get(query_url)
        data.append((response, year))

2019 48 339
2019 48 201
2019 48 015
2019 48 167
2019 48 157
2019 48 071
2019 48 039
2019 48 291
2019 48 473
2020 48 339
2020 48 201
2020 48 015
2020 48 167
2020 48 157
2020 48 071
2020 48 039
2020 48 291
2020 48 473
2021 48 339
2021 48 201
2021 48 015
2021 48 167
2021 48 157
2021 48 071
2021 48 039
2021 48 291
2021 48 473


In [84]:
# # Pickling data 
# pd.to_pickle(data, 'mobility_census_data.pkl')

In [120]:
frame_array = []
for county,year in data:
    j = county.json()
    frame = pd.DataFrame(j[1:], columns=j[0])
    frame['year'] = year
    frame_array.append(frame)

df = pd.concat(frame_array)

,NAME,B01002_001E,B19013_001E,B19301_001E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,state,county,tract,block group,year
0,"Block Group 1, Census Tract 6919, Montgomery C...",36.0,105100,40952,2657,2530,85,0,9,0,0,48,339,691900,1,2019
1,"Block Group 3, Census Tract 6933, Montgomery C...",31.1,54517,34372,3425,3023,220,0,160,0,0,48,339,693300,3,2019
2,"Block Group 1, Census Tract 6933, Montgomery C...",27.6,32863,24291,1619,1409,47,0,46,0,13,48,339,693300,1,2019
3,"Block Group 1, Census Tract 6938, Montgomery C...",32.3,43763,19419,4732,3506,665,0,163,0,308,48,339,693800,1,2019
4,"Block Group 1, Census Tract 6947, Montgomery C...",49.6,85478,51829,1872,1636,185,12,0,0,18,48,339,694700,1,2019


In [122]:
variable_names = {
    'B01002_001E': 'MedAge',
    'B19013_001E': 'MedHouIncome',
    'B19301_001E': 'PerCapIncome',
    'B02001_001E': 'TotPop',
    'B02001_002E': 'White',
    'B02001_003E': 'BlackAA',
    'B02001_004E': 'AIAN',
    'B02001_005E': 'Asian',
    'B02001_006E': 'NHOPI',
    'B02001_007E': 'OtherRace'
}

df.rename(columns=variable_names, inplace=True)

In [123]:
df['GEOID'] = df['state'] + df['county'] + df['tract'] + df['block group']
header_df.STATEFP = header_df.STATEFP.astype(str)
header_df.COUNTYFP = header_df.COUNTYFP.astype(str)
header_df.TRACTCE = header_df.TRACTCE.astype(str)
header_df.BLKGRPCE = header_df.BLKGRPCE.astype(str)
header_df['GEOID'] = header_df['STATEFP'] + header_df['COUNTYFP'] + header_df['TRACTCE'] + header_df['BLKGRPCE']
df.head()

,NAME,MedAge,MedHouIncome,PerCapIncome,TotPop,White,BlackAA,AIAN,Asian,NHOPI,OtherRace,state,county,tract,block group,year,GEOID
0,"Block Group 1, Census Tract 6919, Montgomery C...",36.0,105100,40952,2657,2530,85,0,9,0,0,48,339,691900,1,2019,483396919001
1,"Block Group 3, Census Tract 6933, Montgomery C...",31.1,54517,34372,3425,3023,220,0,160,0,0,48,339,693300,3,2019,483396933003
2,"Block Group 1, Census Tract 6933, Montgomery C...",27.6,32863,24291,1619,1409,47,0,46,0,13,48,339,693300,1,2019,483396933001
3,"Block Group 1, Census Tract 6938, Montgomery C...",32.3,43763,19419,4732,3506,665,0,163,0,308,48,339,693800,1,2019,483396938001
4,"Block Group 1, Census Tract 6947, Montgomery C...",49.6,85478,51829,1872,1636,185,12,0,0,18,48,339,694700,1,2019,483396947001


In [124]:
# Sort dataframe by year in descending order
df_sorted = df.sort_values(by='year', ascending=False)

# Group by block group identifier
grouped_df = df_sorted.groupby(['state', 'county', 'tract', 'block group'])

# Select the first row from each group (most recent year)
most_recent_df = grouped_df.first().reset_index()

# The resulting dataframe (most_recent_df) will contain the combined data
# with only the most recent year for each block group.

In [126]:
# Verify only the most recent info is preserved
print(len(df))
print(df.NAME.nunique())
print(most_recent_df.shape)

11329
5523
(5523, 17)


In [128]:
merged_df = header_df.merge(df, left_on='GEOID', right_on='GEOID')

In [130]:
merged_df.to_csv('../../data/processed_header.csv', index=False)